In [5]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import ot


import pyro
import pyro.contrib.examples.polyphonic_data_loader as poly

# Wassの計算

In [6]:
#　曲数
N = 3

#ドレミ系
def easyTones(N):
    max = 70
    interval = 10
    training_seq_lengths = torch.tensor([8]*N)
    training_data_sequences = torch.zeros(N,8,88)
    for i in range(N):
        training_data_sequences[i][0][int(max-i*interval)  ] = 1
        training_data_sequences[i][1][int(max-i*interval)+2] = 1
        training_data_sequences[i][2][int(max-i*interval)+4] = 1
        training_data_sequences[i][3][int(max-i*interval)+5] = 1
        training_data_sequences[i][4][int(max-i*interval)+7] = 1
        training_data_sequences[i][5][int(max-i*interval)+9] = 1
        training_data_sequences[i][6][int(max-i*interval)+11] = 1
        training_data_sequences[i][7][int(max-i*interval)+12] = 1
    return training_seq_lengths, training_data_sequences

## ドドド、レレレ
def superEasyTones(N):
    training_seq_lengths = torch.tensor([8]*N)
    training_data_sequences = torch.zeros(N,8,88)
    for i in range(N):
        for j in range(8):
            training_data_sequences[i][j][int(30+i*5)] = 1
    return training_seq_lengths, training_data_sequences

## ドドド、ドドド、ドドド
def easiestTones(N):
    training_seq_lengths = torch.tensor([8]*N)
    training_data_sequences = torch.zeros(N,8,88)
    for i in range(N):
        for j in range(8):
            training_data_sequences[i][j][int(70)] = 1
    return training_seq_lengths, training_data_sequences

## クラシック
data = poly.load_data(poly.JSB_CHORALES)
classic_mini_batch = data['train']['sequences'][:N,:8]

## ドレミ系
_,CDE_mini_batch = easyTones(N)

## ドドド、レレレ、ミミミ系
_,DDD_mini_batch = superEasyTones(N)

## ドドドだけ
_,CCC_mini_batch = easiestTones(N)

if classic_mini_batch.size() == CDE_mini_batch.size() and \
   classic_mini_batch.size() == DDD_mini_batch.size() and \
   classic_mini_batch.size() == CCC_mini_batch.size():
   print("SIZE =", classic_mini_batch.size())
   print("torch.Size([曲数, 曲の長さ, 鍵数])")
else :
    assert("ERROR!")

SIZE = torch.Size([3, 8, 88])
torch.Size([曲数, 曲の長さ, 鍵数])


In [7]:
def EMD(training_mini_batch, generated_mini_batch):
    # データの確認
    if training_mini_batch.size() != generated_mini_batch.size():
        assert("ERROR at Wass")

    # numpy バージョンの生成
    trn_np = training_mini_batch.detach().clone().numpy()
    gnt_np = generated_mini_batch.detach().clone().numpy()
    
    # 曲数
    n_songs = training_mini_batch.size()[0]
    # 曲の長さ
    n_tones = training_mini_batch.size()[1]
    # 鍵数
    dim = training_mini_batch.size()[2]
    
    trn_np = trn_np.reshape((n_songs, n_tones * dim))
    gnt_np = gnt_np.reshape((n_songs, n_tones * dim))
    
    # ot の使用
    M = ot.dist(trn_np, gnt_np)
    M /= M.max() #　不要かも
    a, b = np.ones((n_songs,)) / n_songs, np.ones((n_songs,)) / n_songs  # uniform distribution on samples
    gamma = ot.emd(a, b, M)

    # 近似計算の実行
    Tx = []
    for i in range(n_songs):
        ind = gamma[i,:].argmax()
        Tx.append(generated_mini_batch[ind])
    Tx = torch.stack(Tx)
    return torch.norm(Tx-training_mini_batch, dim=1).sum()

In [9]:
print(EMD(classic_mini_batch,classic_mini_batch))
print(EMD(classic_mini_batch,CCC_mini_batch))
print(EMD(CDE_mini_batch,CCC_mini_batch))
print(EMD(DDD_mini_batch,CCC_mini_batch))

tensor(0.)
tensor(61.8017)
tensor(31.3026)
tensor(16.9706)
